In [1]:
import pandas as pd
import re
# Extract and analyze Maruti model names
import pandas as pd
import re
import numpy as np
from collections import Counter
# Load the filtered CSV file
maruti_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/MARUTI.csv')
maruti_data.head()


rc_maker_desc                           rc_maker_model  \
0  MARUTI SUZUKI INDIA LTD.,  MARUTI  TOUR M(O) CNG 1.5L 5MT BSVI-PH2   
1    MARUTI SUZUKI INDIA LTD                MARUTI SWIFT DZIRE ZXI IV   
2    MARUTI SUZUKI INDIA LTD             MARUTI SUZUKI INDIA LTD, STD   
3    MARUTI SUZUKI INDIA LTD                  MARUTHI 800 CAR MPI STD   
4    MARUTI SUZUKI INDIA LTD                              BREZZA ZDI+   

  Mapped Name  
0      MARUTI  
1      MARUTI  
2      MARUTI  
3      MARUTI  
4      MARUTI

In [3]:


# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'MARUTI\s+SUZUKI(\s+INDIA\s+LTD)?[,.]?\s*', '', text)
    text = re.sub(r'MARUTI\s+', '', text)
    text = re.sub(r'MARUTHI\s+', '', text)
    text = re.sub(r'M[\s-]', '', text)  # Remove 'M-' or 'M ' prefix
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
maruti_data['clean_model'] = maruti_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'STD', 'CAR']
filtered_models = maruti_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords:")
for model, count in sorted(model_counts.items(), key=lambda x: x[1], reverse=True)[:30]:
    print(f"{model}: {count}")

# Identify primary model names
maruti_model_keywords = [
    # Passenger vehicles
    "SWIFT", "DZIRE", "ALTO", "BALENO", "WAGON R", "WAGONR", "ERTIGA", "CIAZ",
    "VITARA", "BREZZA", "CELERIO", "IGNIS", "S-CROSS", "RITZ", "ZEN", "ESTILO",
    "A-STAR", "SX4", "EECO", "OMNI", "800", "ALTO K10", "XL6", "S-PRESSO",
    # Variants
    "LXI", "VXI", "ZXI", "DLX", "VDI", "ZDI", "LDI", 
    # Specific identifiers
    "TOUR", "AMT", "CNG", "SHVS", "HYBRID", "AGS", "DDIS",
    # Body types
    "SEDAN", "HATCHBACK", "SUV", "MPV"
]

# Create alias map for common misspellings and variations
maruti_alias_map = {
    # Swift variations
    "SIFT": "SWIFT",
    "SIAFT": "SWIFT",
    "SWIF": "SWIFT",
    "SWIFR": "SWIFT",
    "SWFT": "SWIFT",
    "SWIPT": "SWIFT",
    "SWIT": "SWIFT",
    
    # Dzire variations  
    "DESIRE": "DZIRE",
    "DESIR": "DZIRE",
    "DEZIRE": "DZIRE",
    "DZIR": "DZIRE",
    "DEEIRE": "DZIRE",
    "DIIRE": "DZIRE",
    
    # Alto variations
    "AITO": "ALTO",
    "ALITO": "ALTO",
    "ALLTO": "ALTO",
    "ALOT": "ALTO",
    "ALTIO": "ALTO",
    "ALTRES": "ALTO",
    "ALTTO": "ALTO",
    "ATLO": "ALTO",
    
    # Wagon R variations
    "WAGNR": "WAGON R",
    "WAGEN": "WAGON R",
    "WAGANR": "WAGON R",
    "WAGENR": "WAGON R",
    "WAGNO": "WAGON R",
    "WAGNOR": "WAGON R",
    "VAGON": "WAGON R",
    "WAGEN R": "WAGON R",
    
    # Baleno variations
    "BALANO": "BALENO",
    "BALENA": "BALENO",
    "BALINO": "BALENO",
    "BELENO": "BALENO",
    "BALEND": "BALENO",
    
    # Ertiga variations
    "ERITGA": "ERTIGA",
    "ERTGA": "ERTIGA",
    "ETRIGA": "ERTIGA",
    "ERTGIA": "ERTIGA",
    "ERTIG": "ERTIGA",
    
    # Ciaz variations
    "CAIZ": "CIAZ",
    "CIAZ": "CIAZ",
    "CIAZZ": "CIAZ",
    
    # Brezza variations
    "BREZA": "BREZZA",
    "BREEZA": "BREZZA",
    "BREEZ": "BREZZA",
    "BREZZAA": "BREZZA",
    "BREEZA": "BREZZA",
    
    # Celerio variations
    "CELEIO": "CELERIO",
    "CELERO": "CELERIO",
    "CELERLO": "CELERIO",
    "CELEROO": "CELERIO",
    "CELERRIO": "CELERIO",
    "CELERL": "CELERIO",
    "CELLERIO": "CELERIO",
    
    # Vitara variations
    "VETARA": "VITARA",
    "VITAAR": "VITARA",
    "VITANA": "VITARA",
    "VITARE": "VITARA",
    "VETARA": "VITARA",
    
    # Ritz variations
    "RIST": "RITZ",
    "RISTZ": "RITZ",
    "RITIZ": "RITZ",
    "RITJ": "RITZ",
    "RITZ-": "RITZ",
    "RIZTZ": "RITZ",
    
    # Omni variations
    "OMINI": "OMNI",
    "OMINI": "OMNI",
    "OMNE": "OMNI",
    "OMANI": "OMNI",
    "OMINI": "OMNI",
    "OMNY": "OMNI",
    "OMMI": "OMNI",
    
    # Combined models
    "K10": "ALTO K10",
    
    # Trims and Variants
    "LXI+": "LXI",
    "VXI+": "VXI",
    "ZXI+": "ZXI",
    "VDI+": "VDI",
    "ZDI+": "ZDI",
}

# Function to extract Maruti model keywords
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

def extract_maruti_model(text):
    txt = normalize(text)
    # First, correct aliases
    for bad, good in maruti_alias_map.items():
        txt = txt.replace(bad, good)
    # Then search for any keyword
    for kw in sorted(maruti_model_keywords, key=len, reverse=True):
        if kw in txt:
            return kw
    return "NOT FOUND"

# Apply extraction to the entire dataset
print("Applying extraction to all", len(maruti_data), "rows...")
maruti_data['maruti_model'] = maruti_data['rc_maker_model'].apply(extract_maruti_model)

# Create a final_model column using fallback logic
maruti_data['final_model'] = maruti_data['maruti_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Display extraction statistics
model_detection = maruti_data['maruti_model'].value_counts()
print("\nTop 30 detected models:")
print(model_detection.head(30))

print(f"\nDetection rate: {(maruti_data['maruti_model'] != 'NOT FOUND').sum()} out of {len(maruti_data)} ({(maruti_data['maruti_model'] != 'NOT FOUND').sum()/len(maruti_data)*100:.2f}%)")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/maruti_models_extracted.csv'
maruti_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = maruti_data.groupby('rc_maker_model')['maruti_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(maruti_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/maruti_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

Top model keywords:
VXI: 2440
LXI: 2020
SWIFT: 1907
ALTO: 1647
R: 1156
WAGON: 1099
DZIRE: 1066
VDI: 1061
BS4: 1042
O: 1007
ZXI: 993
ZEN: 920
1: 820
ZDI: 719
LX: 715
CIAZ: 621
EECO: 600
M: 585
800: 574
ERTIGA: 565
OMNI: 556
AMT: 553
BALENO: 543
5: 501
CELERIO: 482
AC: 477
BREZZA: 469
5MT: 444
BS3: 428
SMART: 408
Applying extraction to all 24724 rows...

Top 30 detected models:
maruti_model
SWIFT        3962
ALTO         2725
NOT FOUND    2202
WAGON R      1732
OMNI         1483
800          1196
ERTIGA       1033
EECO          928
BALENO        834
ZEN           795
CELERIO       789
CIAZ          787
DZIRE         733
ALTO K10      633
VITARA        622
RITZ          600
ESTILO        584
WAGONR        567
VXI           506
SX4           395
LXI           328
IGNIS         327
HYBRID        272
TOUR          244
BREZZA        173
ZXI           116
VDI            40
ZDI            26
CNG            24
XL6            21
Name: count, dtype: int64

Detection rate: 22522 out of 24724 (91.09